In [5]:
import pandas as pd
import dotenv
import os
import boto3

In [38]:
dotenv.load_dotenv(dotenv.find_dotenv())


key_id = os.getenv("aws_access_key_id")
key_acess = os.getenv("aws_secret_access_key")
aws_token = os.getenv("aws_session_token")



s3 = boto3.client(
    's3',
    aws_access_key_id=key_id,
    aws_secret_access_key=key_acess,
    aws_session_token=aws_token
)



bucket_name = 'raw-analise-voos-grupo2'
pasta_local = 'dados/raw'

In [ ]:
def upload_s3():
    for arquivo in os.listdir(pasta_local):
        caminho_arquivo = os.path.join(pasta_local, arquivo)
        
        try:
            s3.upload_file(caminho_arquivo, bucket_name, arquivo)
            print(f"Arquivo {arquivo} enviado para o bucket {bucket_name} com sucesso!")
            
            os.remove(caminho_arquivo)
            print(f"Arquivo {arquivo} apagado da pasta local.")
        
        except Exception as e:
            print(f"Erro ao enviar ou apagar o arquivo {arquivo}: {e}")
            
upload_s3()